## Bank Marketing Analysis

by Runtian Li, Rafe Chang, Sid Grover, Anu Banga

**Repo Link:** https://github.com/UBC-MDS/dsci_522_group_8.git

In [1]:
## Import necessary Packages
import altair as alt
import altair_viewer
import pandas as pd
import numpy as np
import statistics
import os
import sys

sys.path.append("code/.")

# Data 
from ucimlrepo import fetch_ucirepo 

# Machine Learning
import IPython
import matplotlib.pyplot as plt
import mglearn
from IPython.display import HTML, display
# from plotting_functions import *
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# %matplotlib inline
pd.set_option("display.max_colwidth", 200)

from IPython.display import Image

# Summary
TBD

# Introduction
TBD

# Method
## Data
The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. It was sourced from the UCI Machine Learning Repository and can be found [here](https://archive.ics.uci.edu/dataset/222/bank+marketing). We will be using bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs). 

These are the detail of all inputs:

| Feature Name | Type        | Description                                                                                   | Classes |
|--------------|-------------|-----------------------------------------------------------------------------------------------|---------|
| age          | Numeric     |                                                                                               |         |
| job          | Categorical | Type of job                                                                                   | 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital      | Categorical | Marital status                                                                                | 'divorced','married','single','unknown' |
| education    | Categorical |                                                                                               | 'primary', 'secondary', 'tertiary', 'unknown' |
| default      | Categorical | Has credit in default?                                                                        | 'no', 'yes', 'unknown' |
| housing      | Categorical | Has housing loan?                                                                             | 'no', 'yes', 'unknown' |
| loan         | Categorical | Has personal loan?                                                                            | 'no', 'yes', 'unknown' |
| balance      | Numeric     | Balance of the individual                                                                     |         |
| contact      | Categorical | Contact communication type                                                                    | 'cellular', 'telephone' |
| month        | Categorical | Last contact month of year                                                                    | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day          | Categorical | Last contact day of the week                                                                  | 'mon', 'tue', 'wed', 'thu', 'fri' |
| duration     | Numeric     | Last contact duration, in seconds                                                             |         |
| campaign     | Numeric     | Number of contacts performed during this campaign and for this client                        |         |
| pdays        | Numeric     | Number of days that passed by after the client was last contacted from a previous campaign    |         |
| previous     | Numeric     | Number of contacts performed before this campaign and for this client                         |         |
| poutcome     | Categorical | Outcome of the previous marketing campaign                                                    | 'failure', 'nonexistent', 'success' |
| y            | Binary      | Has the client subscribed to a term deposit?                                                  | 'yes', 'no' |


The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).

## Analysis
TBD

# Results and Discussion

TBD

### Exploratory Data Analysis 

In [2]:
# # fetch dataset 
# bank_marketing = fetch_ucirepo(id=222) 
  
# # data (as pd dataframes) 
# X = bank_marketing.data.features 
# y = bank_marketing.data.targets 
  
# # metadata 
# #display(bank_marketing.metadata) 
  
# # variable information 
# display(bank_marketing.variables) 
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

df = pd.read_csv("data/bank-full.csv", delimiter=";")
df.rename(columns={"y": "target"}, inplace=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36168 entries, 28686 to 15725
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        36168 non-null  int64 
 1   job        36168 non-null  object
 2   marital    36168 non-null  object
 3   education  36168 non-null  object
 4   default    36168 non-null  object
 5   balance    36168 non-null  int64 
 6   housing    36168 non-null  object
 7   loan       36168 non-null  object
 8   contact    36168 non-null  object
 9   day        36168 non-null  int64 
 10  month      36168 non-null  object
 11  duration   36168 non-null  int64 
 12  campaign   36168 non-null  int64 
 13  pdays      36168 non-null  int64 
 14  previous   36168 non-null  int64 
 15  poutcome   36168 non-null  object
 16  target     36168 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.0+ MB


In [4]:
display(pd.DataFrame(train_df.describe()).T)

,count,mean,std,min,25%,50%,75%,max
age,36168.0,40.944979,10.609908,18.0,33.0,39.0,48.00,95.0
balance,36168.0,1371.354208,2999.155128,-8019.0,73.0,448.5,1448.00,98417.0
day,36168.0,15.801095,8.309679,1.0,8.0,16.0,21.00,31.0
duration,36168.0,258.955403,259.218884,0.0,103.0,180.0,319.25,4918.0
campaign,36168.0,2.759013,3.095290,1.0,1.0,2.0,3.00,58.0
pdays,36168.0,40.199762,100.114274,-1.0,-1.0,-1.0,-1.00,871.0
previous,36168.0,0.580596,2.364362,0.0,0.0,0.0,0.00,275.0


In [3]:
train_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,target
28686,29,services,single,secondary,no,-205,no,no,cellular,30,jan,194,1,-1,0,unknown,no
9304,53,blue-collar,married,primary,no,0,yes,no,unknown,5,jun,285,2,-1,0,unknown,no
41425,55,management,married,primary,no,2587,no,no,cellular,4,sep,328,2,94,2,success,yes
44803,30,technician,single,tertiary,no,0,no,no,cellular,15,sep,214,2,-1,0,unknown,no
5878,30,unemployed,married,secondary,no,529,yes,yes,unknown,26,may,107,3,-1,0,unknown,no


In [6]:
# #Distribution
# numeric = ["age", "balance", "day_of_week", "duration", "campaign", "pdays", "previous"]
# alt.data_transformers.enable("vegafusion")

# charts = []

# # Create density plots for each numeric column
# for column in numeric:
#     chart = alt.Chart(df).transform_density(
#         column,
#         groupby=["target"],  # Assuming "target" is a column in your DataFrame
#         as_=[column, "density"]
#     ).mark_area(opacity=0.5).encode(
#         x=alt.X(f"{column}:Q").scale(zero = False),
#         y=alt.Y("density:Q").stack(False).scale(zero = False),
#         color="target"
#     )
#     charts.append(chart)

In [7]:
# for i in range(0,len(charts)-1):
#     display(charts[i])

In [8]:
# alt.Chart(df).mark_bar().encode(
#     y = alt.Y(alt.repeat()).sort("x"),
#     x = alt.X("count()").stack(False).scale(zero = False),
#     color = alt.Color('target:O')
# ).repeat(["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome"])

In [4]:
alt.data_transformers.enable("vegafusion")

numerical_col = train_df.select_dtypes(include=['int64', 'float64']).columns.to_list()

# Create histograms for each numerical column
numeric_cols_hists = alt.Chart(train_df).mark_bar().encode(
    alt.X(alt.repeat("repeat"), type="quantitative", bin=alt.Bin(maxbins=50)),
    y="count()",
).properties(
    width=300,
    height=200
).repeat(
    repeat=numerical_col,
    columns=1  # Adjust the number of columns based on your preference
)

numeric_cols_hists

alt.RepeatChart(...)

In [10]:
# Correlation matrix

correlation_matrix = train_df[numerical_col].corr('spearman').style.background_gradient()
correlation_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.098459,-0.010320,-0.032978,0.036050,-0.018972,-0.013098
balance,0.098459,1.000000,0.003357,0.045656,-0.031664,0.069799,0.079347
day,-0.010320,0.003357,1.000000,-0.054977,0.139057,-0.088811,-0.084349
duration,-0.032978,0.045656,-0.054977,1.000000,-0.108825,0.023665,0.026642
campaign,0.036050,-0.031664,0.139057,-0.108825,1.000000,-0.106710,-0.102925
pdays,-0.018972,0.069799,-0.088811,0.023665,-0.106710,1.000000,0.985654
previous,-0.013098,0.079347,-0.084349,0.026642,-0.102925,0.985654,1.000000


In [5]:
# Distribution of categorical columns
categorical_chart = alt.Chart(train_df).mark_bar().encode(
    x=alt.X('count()', title='Count'),
    y=alt.Y(alt.repeat("row"), type="nominal", sort='-x'),
).properties(
    width=300,
    height=200
).repeat(
    row = ["job", "marital", "education", "default", "housing", "loan", "poutcome"]
)
categorical_chart

alt.RepeatChart(...)

### Preprocessing

<div class="alert alert-info">
    
- Since there is no missing values in our dataset, we don't need to do imputation or drop NAs.   
- We are going to drop "contact", "day" and "month" column here since they are not helping us in identifying useful underlying pattern in the model.    
- We take "age", "balance", "duration", "campaign", "pdays", "previous" as numerical features and we are doing StandardScaler transformation on them.
- We take "job", "marital", "education", "default", "housing", "loan", "poutcome" as categorical features and we are doing one hot encoding on them. We dropped columns only if the categorical is binary.
    
</div>

In [6]:
numeric_looking_columns = train_df.select_dtypes(include=np.number).columns.tolist()
print(numeric_looking_columns)

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [7]:
# Lists of feature names
numerical_features = ["age", "balance", "duration", "campaign", "pdays", "previous"]
categorical_features = ["job", "marital", "education", "default", "housing", "loan", "poutcome"]
drop_features = ["contact", "day", "month"]

# Create the column transformer
preprocessor = make_column_transformer(    
    (StandardScaler(), numerical_features),  # scaling on numeric features   
    (OneHotEncoder(drop="if_binary"), categorical_features),  # OHE on categorical features
    ("drop", drop_features),  # drop the drop features
)

# Show the preprocessor
preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['age', 'balance', 'duration', 'campaign',
                                  'pdays', 'previous']),
                                ('onehotencoder',
                                 OneHotEncoder(drop='if_binary'),
                                 ['job', 'marital', 'education', 'default',
                                  'housing', 'loan', 'poutcome']),
                                ('drop', 'drop', ['contact', 'day', 'month'])])

In [8]:
# Seperate X and y
X_train = train_df.drop(columns=["target"])
X_test = test_df.drop(columns=["target"])
y_train = train_df["target"]
y_test = test_df["target"]

In [9]:
# This line nicely formats the feature names from `preprocessor.get_feature_names_out()` so that we can more easily use them below
preprocessor.verbose_feature_names_out = False

# Create a dataframe with the transformed features and column names
ct = preprocessor.fit(X_train)

# Columns names after one hot encoding
ohe_columns = list(
    preprocessor.named_transformers_["onehotencoder"]
    .get_feature_names_out(categorical_features)
)

# Columns after transformation
new_columns = (
    numerical_features + ohe_columns
)

# Now create the DataFrame with the dense data
X_train_enc = pd.DataFrame(preprocessor.transform(X_train), index=X_train.index, columns=new_columns)
X_train_enc.head()

,age,balance,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,education_secondary,education_tertiary,education_unknown,default_yes,housing_yes,loan_yes,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
28686,-1.125848,-0.525607,-0.250585,-0.568295,-0.411533,-0.245565,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9304,1.136220,-0.457253,0.100475,-0.245219,-0.411533,-0.245565,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
41425,1.324725,0.405335,0.266360,-0.245219,0.537396,0.600341,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44803,-1.031595,-0.457253,-0.173429,-0.245219,-0.411533,-0.245565,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5878,-1.031595,-0.280868,-0.586213,0.077857,-0.411533,-0.245565,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


### Model Selection

In [10]:
# 1. Base Model: Dummy Classifier
classification_metrics = ["accuracy", "precision", "recall", "f1"]
dc = DummyClassifier(strategy="most_frequent")
pipe_dc = make_pipeline(preprocessor, dc)
# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results = {}
scoring = {
    "accuracy": 'accuracy',
    'precision': make_scorer(precision_score, pos_label="yes", zero_division=0),
    'recall': make_scorer(recall_score, pos_label="yes"),
    'f1': make_scorer(f1_score, pos_label="yes")
}  # scoring can be a string, a list, or a dictionary

cross_val_results['dummy'] = pd.DataFrame(cross_validate(pipe_dc, X_train, y_train, return_train_score=True, scoring=scoring)).agg(['mean', 'std']).round(3).T

# Show the train and validation scores
cross_val_results['dummy']

,mean,std
fit_time,0.179,0.009
score_time,0.314,0.023
test_accuracy,0.883,0.000
train_accuracy,0.883,0.000
test_precision,0.000,0.000
train_precision,0.000,0.000
test_recall,0.000,0.000
train_recall,0.000,0.000
test_f1,0.000,0.000
train_f1,0.000,0.000


In [17]:
# 2. Logistic regression

# The logreg model pipeline
logreg = make_pipeline(preprocessor, LogisticRegression(max_iter=1000, random_state=123))

# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results['logreg'] = pd.DataFrame(cross_validate(logreg, X_train, y_train, return_train_score=True, scoring=scoring)).agg(['mean', 'std']).round(3).T

# Show the train and validation scores
cross_val_results['logreg'] 

,mean,std
fit_time,0.617,0.015
score_time,0.386,0.027
test_accuracy,0.900,0.003
train_accuracy,0.900,0.001
test_precision,0.652,0.029
train_precision,0.655,0.006
test_recall,0.313,0.019
train_recall,0.315,0.009
test_f1,0.423,0.023
train_f1,0.425,0.009


In [18]:
# 3. Support vector classifier

# The svc model pipeline
svc = make_pipeline(preprocessor, SVC(random_state=123))

# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results['svc'] = pd.DataFrame(cross_validate(svc, X_train, y_train, return_train_score=True, scoring=scoring)).agg(['mean', 'std']).round(3).T
# Show the train and validation scores
cross_val_results['svc'] 

,mean,std
fit_time,23.607,0.769
score_time,8.785,0.035
test_accuracy,0.899,0.002
train_accuracy,0.907,0.001
test_precision,0.655,0.016
train_precision,0.726,0.007
test_recall,0.288,0.008
train_recall,0.326,0.007
test_f1,0.400,0.010
train_f1,0.450,0.007


In [19]:
# 4. Balanced logistic regression
logreg_bal = make_pipeline(preprocessor, 
                           LogisticRegression(max_iter=1000, 
                                              random_state=123, 
                                              class_weight="balanced"))

# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results['logreg_bal'] = pd.DataFrame(cross_validate(logreg_bal, X_train, y_train, return_train_score=True, scoring=scoring)).agg(['mean', 'std']).round(3).T

# Show the train and validation scores
cross_val_results['logreg_bal'] 

,mean,std
fit_time,0.563,0.047
score_time,0.368,0.022
test_accuracy,0.829,0.002
train_accuracy,0.829,0.001
test_precision,0.386,0.005
train_precision,0.386,0.003
test_recall,0.777,0.012
train_recall,0.778,0.002
test_f1,0.516,0.006
train_f1,0.516,0.003


In [20]:
# 5. Balanced support vector classifier
svc_bal = make_pipeline(preprocessor, SVC(random_state=123, class_weight="balanced"))

# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results['svc_bal'] = pd.DataFrame(cross_validate(svc_bal, X_train, y_train, return_train_score=True, scoring=scoring)).agg(['mean', 'std']).round(3).T

# Show the train and validation scores
cross_val_results['svc_bal'] 

,mean,std
fit_time,56.586,32.579
score_time,16.147,0.719
test_accuracy,0.814,0.006
train_accuracy,0.825,0.001
test_precision,0.368,0.010
train_precision,0.388,0.001
test_recall,0.821,0.011
train_recall,0.864,0.004
test_f1,0.508,0.011
train_f1,0.535,0.001


In [21]:
# Compare the average scores of all the models
pd.concat(
    cross_val_results,
    axis='columns'
).xs(
    'mean',
    axis='columns',
    
    level=1
).style.format(
    precision=2
).background_gradient(
    axis=None
)

,dummy,logreg,svc,logreg_bal,svc_bal
fit_time,0.18,0.62,23.61,0.56,56.59
score_time,0.31,0.39,8.79,0.37,16.15
test_accuracy,0.88,0.90,0.90,0.83,0.81
train_accuracy,0.88,0.90,0.91,0.83,0.82
test_precision,0.00,0.65,0.66,0.39,0.37
train_precision,0.00,0.66,0.73,0.39,0.39
test_recall,0.00,0.31,0.29,0.78,0.82
train_recall,0.00,0.32,0.33,0.78,0.86
test_f1,0.00,0.42,0.40,0.52,0.51
train_f1,0.00,0.42,0.45,0.52,0.54


`Dummy Classifier` has low accuracy and zero precision, recall, and F1 scores, indicating it never predicts the positive class (in this case the client subscribed a term deposit). This is expected as it always predicts the most frequent class.

`logreg` shows improved accuracy over the dummy model. However, its recall is low, suggesting it misses a significant number of true positive cases. `svc` performed almost the same as logistic regression model among all metrics.

`logreg_bal` and `svc_bal` have lower accuracy compared to their unbalanced counterparts but significantly higher recall. This indicates they are better at identifying positive cases but at the cost of making more false positive errors.

Given the context of our bank marketing data set, we aim to detect the clients who will subscribe a term deposit given the features. Missing a potential "yes" could be more costly than false positives, as it represents a lost opportunity for the sales team to transform this potential customer. Therefore, we chose `svc_bal` as the model has the highest `test_recall` score. 

In [12]:
# Creating a sample of 10000 observations
sample_data = df.sample(n=10000, random_state=123)
train_df_sampled, test_df_sampled = train_test_split(sample_data, test_size=0.2, random_state=123)

X_train_sampled = train_df_sampled.drop(columns=["target"])
X_test_sampled = test_df_sampled.drop(columns=["target"])
y_train_sampled = train_df_sampled["target"]
y_test_sampled = test_df_sampled["target"]

# Transformation on the sample training data
sample_preprocessor = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(drop="if_binary"), categorical_features),
    ("drop", drop_features),
)

X_train_sampled_enc = pd.DataFrame(sample_preprocessor.fit_transform(X_train_sampled), index=X_train_sampled.index, columns=new_columns)

svc_bal_sample = make_pipeline(sample_preprocessor, SVC(random_state=123, class_weight="balanced"))

param_dist = {
    'svc__C': uniform(0.1, 10),
    'svc__gamma': uniform(0.001, 0.1),
    'svc__kernel': ['rbf', 'sigmoid', 'linear']
}

# Perform RandomizedSearchCV for hyperparameter optimization
random_search = RandomizedSearchCV(svc_bal_sample, param_distributions=param_dist, n_iter=25, cv=5, n_jobs=-1, random_state=123)
random_search.fit(X_train_sampled, y_train_sampled)

# Best hyperparameters
best_params_random = random_search.best_params_
print("Best Hyperparameters (Randomized Search):", best_params_random)


Best Hyperparameters (Randomized Search): {'svc__C': 4.331064601244609, 'svc__gamma': 0.09907641983846155, 'svc__kernel': 'rbf'}


In [13]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "param_svc__gamma",
        "param_svc__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T


rank_test_score,1,2,3,3,3,3,7,7,9,10,...,16,17,18,19,20,21,22,23,24,25
mean_test_score,0.831875,0.82775,0.8275,0.8275,0.8275,0.8275,0.827375,0.827375,0.825,0.82125,...,0.819,0.819,0.818375,0.76425,0.742125,0.73775,0.732375,0.730375,0.71675,0.714125
param_svc__gamma,0.099076,0.008709,0.044086,0.073905,0.069326,0.018537,0.072915,0.012562,0.067931,0.054183,...,0.044342,0.040804,0.025367,0.023826,0.042702,0.042721,0.049303,0.052176,0.0776,0.083634
param_svc__C,4.331065,1.640822,4.437012,3.53178,5.073088,5.40062,7.008849,6.33953,5.204223,5.415514,...,2.927029,0.696779,7.736828,3.717887,3.141208,3.677289,2.604554,7.034032,9.979953,1.306287
mean_fit_time,10.371298,13.673076,26.596402,23.437286,27.853274,20.952369,29.367206,32.64497,12.097913,11.799298,...,12.345532,12.501173,10.268341,7.618212,7.385023,8.132923,8.672542,7.734671,6.945047,7.717469


In [14]:
# Evaluate the best model on the test set
best_model_random = random_search.best_estimator_
accuracy_random = best_model_random.score(X_test_sampled, y_test_sampled)
print("Accuracy on Test Set:", accuracy_random)


Accuracy on Test Set: 0.825


In [15]:
results = pd.DataFrame(random_search.cv_results_)

scatter = alt.Chart(results).mark_circle().encode(
    x='param_svc__C:Q',
    y='param_svc__gamma:Q',
    color=alt.Color('mean_test_score:Q', 
                    scale=alt.Scale(scheme='viridis', reverse=True)
                   )
).properties(
    width=400,
    height=300,
    title='C and gamma vs. Mean Test Score'
)

scatter


alt.Chart(...)

The prediction model demonstrated good performance on the test data, achieving an overall accuracy of 82.5%. This suggests that nearly 83 out of every 100 predictions made by the model aligned with the actual outcomes in the test dataset.

Optimizing hyperparameters in Support Vector Machines (SVM) with a smaller sample size of 10,000 instances is a strategy aimed at enhancing computational efficiency. This approach expedites the exploration of hyperparameter possibilities, aiding in the discovery of potential configurations. While the outcomes validate the concept, it's crucial to recognize and manage the constraints stemming from the smaller dataset size when interpreting the results.


# References